In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.callbacks import get_openai_callback
template = PromptTemplate.from_template

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()] #LLM의 event에 반응함
)
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}"
# )

# prompt = template.format(country_a="Mexico", country_b="Thailand")


# chat.predict(prompt)

In [10]:
template = ChatPromptTemplate.from_messages([
    ("system","너는 지질학 전문가고 무조껀 {language}로 대답해"),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human", "{country_a}와 {country_b} 사이의 거리는 어떻게 되나요? 그리고 너의 이름은 뭐야?")
])

prompt = template.format_messages(
    language="japanese",
    name="Socrates",
    country_a="Mexio",
    country_b="seoul"
)

chat.predict_messages(prompt)

AIMessage(content='私の名前はソクラテスです。メキシコシティとソウル 사이의 거리는 약 12,000km 정도입니다.')

In [13]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    

p = CommaOutputParser()

p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [20]:
template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a list generating Machine. EveryThing you are asked will be answered with a comma separateed list of max {max_items}. Do Not reply with anything else"),
        ("human","{question}")        
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets?"
)
result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [22]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the Pokemons? korean plz"
})

['피카츄', '꼬부기', '파이리', '이상해씨', '푸린']

In [23]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

In [28]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipes vegeterian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare them as substitutes in this recipe:

1. **Tofu**: 
   - Use firm or extra-firm tofu for this recipe.
   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.
   - Cut the tofu into bite-sized cubes and follow the same marination process as you would with the chicken. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.
   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy.

2. **Paneer**:
   - Paneer is a type of Indian cheese that holds its shape well when cooked.
   - Cut the paneer into bite-sized cubes and follow the same marination process as you would with the chicken.
   - Instead of ba

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare them as substitutes in this recipe:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marination process as you would with the chicken. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked.\n   - Cut the paneer into bite-sized cubes and follow the same marination process as you would wi

In [33]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})


AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

In [39]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}? plz answer korean"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        Capital: 방콕 (Bangkok)
        Language: 태국어 (Thai)
        Food: 팟타이 (Pad Thai) and 타이 커리 (Thai Curry)
        Currency: 태국 바트 (Thai Baht)

AIMessageChunk(content='\n        Capital: 방콕 (Bangkok)\n        Language: 태국어 (Thai)\n        Food: 팟타이 (Pad Thai) and 타이 커리 (Thai Curry)\n        Currency: 태국 바트 (Thai Baht)')

In [48]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [63]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "릴파",
        "example_question": "너의 지역은 어디야?",
        "example_answer": "리라리라~~",
        "question": "너가 제일 좋아하는 음식은 뭐야?",
    }
)

릴라릴라~ 나는 과일을 좋아해! 특히 딸기와 바나나가 제일 좋아해!

AIMessageChunk(content='릴라릴라~ 나는 과일을 좋아해! 특히 딸기와 바나나가 제일 좋아해!')

In [64]:
set_llm_cache(SQLiteCache("cache.db"))

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [ ]:
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)